In [4]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import math
import time


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [5]:
class AdaptiveRelu(nn.Module):
    def __init__(self):
        super(AdaptiveRelu, self).__init__()
        self.thr = nn.Parameter(torch.tensor(0.0))

    def forward(self, x):
        x = nn.LeakyReLU()(x - self.thr) + self.thr
        return x

In [16]:
import torch.nn as nn
from torch.nn import Sequential


class Flatten(nn.Module):
    """
    Implement a simple custom module that reshapes (n, m, 1, 1) tensors to (n, m).
    """
    def forward(self,x):
        n,m,i,j = x.size()
        return x.view(n,m)
        

def all_cnn_module():
    """
    Create a nn.Sequential model containing all of the layers of the All-CNN-C as specified in the paper.
    https://arxiv.org/pdf/1412.6806.pdf
    Use a AvgPool2d to pool and then your Flatten layer as your final layers.
    You should have a total of exactly 23 layers of types:
    - nn.Dropout
    - nn.Conv2d
    - nn.ReLU
    - nn.AvgPool2d
    - Flatten
    :return: a nn.Sequential model
    """
    return Sequential(
            nn.Dropout(0.2),
            nn.Conv2d(3,96,(3,3),stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(96,96,(3,3),stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(96,96,(3,3),stride=(2, 2), padding=(1, 1)),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv2d(96,192,(3,3),stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(192,192,(3,3),stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.Conv2d(192,192,(3,3),stride=(2, 2), padding=(1, 1)),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv2d(192,192,(3,3),stride=(1,1), padding=(0, 0)),
            nn.ReLU(),
            nn.Conv2d(192,192,(1,1),stride=(1, 1), padding=(0, 0)),
            nn.ReLU(),
            nn.Conv2d(192,10,(1,1),stride=(1, 1), padding=(0, 0)),
            nn.ReLU(),
            nn.AvgPool2d(6,6),
            Flatten()
            )


In [17]:
import numpy as np

def sample_zero_mean(x):
    """
    Make each sample have a mean of zero by subtracting mean along the feature axis.
    :param x: float32(shape=(samples, features))
    :return: array same shape as x
    """
    x = x.T - np.mean(x, axis=1)
    return x.T

def gcn(x, scale=55., bias=0.01):
    """
    GCN each sample (assume sample mean=0)
    :param x: float32(shape=(samples, features))
    :param scale: factor to scale output
    :param bias: bias for sqrt
    :return: scale * x / sqrt(bias + sample variance)
    """
    var = np.var(x, axis=1)
    GCN = scale * x.T / np.sqrt(bias + var)
    return GCN.T


def feature_zero_mean(x, xtest):
    """
    Make each feature have a mean of zero by subtracting mean along sample axis.
    Use train statistics to normalize test data.
    :param x: float32(shape=(samples, features))
    :param xtest: float32(shape=(samples, features))
    :return: tuple (x, xtest)
    """
    mean = np.mean(x, axis = 0)
    x = x - mean
    xtest = xtest - mean
    return x, xtest


def zca(x, xtest, bias=0.1):
    """
    ZCA training data. Use train statistics to normalize test data.
    :param x: float32(shape=(samples, features)) (assume mean=0)
    :param xtest: float32(shape=(samples, features))
    :param bias: bias to add to covariance matrix
    :return: tuple (x, xtest)
    """
    d,n = x.shape
    cov = np.dot(x.T,x)/d + np.eye(n)*bias
    print(cov.shape)
    U,S,V = np.linalg.svd(cov)
    S_1 = 1./S * np.eye(n)
    W = np.dot( np.dot(U, np.sqrt(S_1)) , V)
    return np.dot(x,W),np.dot(xtest,W)


def cifar_10_preprocess(x, xtest, image_size=32):
    """
    1) sample_zero_mean and gcn xtrain and xtest.
    2) feature_zero_mean xtrain and xtest.
    3) zca xtrain and xtest.
    4) reshape xtrain and xtest into NCHW
    :param x: float32 flat images (n, 3*image_size^2)
    :param xtest float32 flat images (n, 3*image_size^2)
    :param image_size: height and width of image
    :return: tuple (new x, new xtest), each shaped (n, 3, image_size, image_size)
    """
    x = sample_zero_mean(x)
    xtest = sample_zero_mean(xtest)
    x = gcn(x)
    xtest = gcn(xtest)
    x, xtest = feature_zero_mean(x, xtest)
    x, xtest = zca(x, xtest)
    
    return x.reshape(-1,3, image_size, image_size),xtest.reshape(-1,3, image_size, image_size)
        


In [24]:
"""
Refer to handout for details.
- Build scripts to train your model
- Submit your code to Autolab
"""
import numpy as np
import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import TensorDataset


def loading_data():
    x_train = np.load('train_feats.npy')
    x_test = np.load('test_feats.npy')
    y_train = np.load('train_labels.npy')
    print(x_train.shape,x_test.shape,y_train.shape)
    return x_train, y_train, x_test
    
def initializer(module):
    if hasattr(module, 'bias'):
        module.bias.data.zero_()
    if hasattr(module, 'weight'):
        torch.nn.init.xavier_uniform(module.weight.data)

def write_results(predictions, output_file='predictions.txt'):
    """
    Write predictions to file for submission.
    File should be:
        named 'predictions.txt'
        in the root of your tar file
    :param predictions: iterable of integers
    :param output_file:  path to output file.
    :return: None
    """
    with open(output_file, 'w') as f:
        for y in predictions:
            f.write("{}\n".format(y))
            
def to_variable(tensor):
    if torch.cuda.is_available():
        tensor = tensor.cuda()
    return torch.autograd.Variable(tensor)

def main():
    x_train, y_train, x_test = loading_data()
    x_train, x_test = cifar_10_preprocess(x_train, x_test, image_size=32)
    print(x_train.shape)
    print(x_test.shape)
    print(y_train[1:10])
    idx = np.arange(len(x_train))
    np.random.shuffle(idx)
    x_train = x_train[idx]
    y_train = y_train[idx]
    y_test = np.zeros(len(x_test))
    x,y = x_train[1:45000],y_train[1:45000]
    x_val,y_val = x_train[45000:],y_train[45000:]
    #x, x_val = p.cifar_10_preprocess(x, x_val, image_size=32)
    train_loader = DataLoader(TensorDataset(torch.from_numpy(x),torch.from_numpy(y)),batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(torch.from_numpy(x_val),torch.from_numpy(y_val)),batch_size=32, shuffle=False)
    test_loader = DataLoader(TensorDataset(torch.from_numpy(x_test),torch.from_numpy(y_test)),batch_size=32, shuffle=False)
    my_net = all_cnn_module()
    #my_net.apply(initializer)
    my_net.load_state_dict(torch.load("check1.py"))
    my_net.double()
    
    loss_fn = torch.nn.CrossEntropyLoss()
    optim = torch.optim.SGD(my_net.parameters(), lr=0.0001, momentum=0.9,weight_decay=0.00001)
    if torch.cuda.is_available():
        my_net = my_net.cuda()
        loss_fn = loss_fn.cuda()
    toplot = []
    toplot1 = []
    for epoch in range(20):
        losses = []
        my_net.train()
        for (input_val, label) in train_loader:
            optim.zero_grad()  # Reset the gradients
            #input_val = to_tensor(input_val)
            #label = to_tensor(label)
            prediction = my_net(to_variable(input_val))  # Feed forward
            loss = loss_fn(prediction, to_variable(label.long()))  # Compute losses
            loss.backward()  # Backpropagate the gradients
            losses.append(loss.data.cpu().numpy())
            optim.step()  # Update the network
        print("Epoch {} Loss: {:.4f}".format(epoch, np.asscalar(np.mean(losses))))
        
        my_net.eval()
        pred5 = []
        
        for (input_val, label) in val_loader:
            prediction = my_net(to_variable(input_val))
            pred5.extend(prediction.data.cpu().numpy())
        print(": ",(np.argmax(pred5, axis=1)==y_val).mean())
        toplot.append((np.argmax(pred5, axis=1)==y_val).mean())
        toplot1.append(np.asscalar(np.mean(losses)))
        name1 = "accuracy_{}: {}, {}".format(epoch, (np.argmax(pred5, axis=1)==y_val).mean(), np.asscalar(np.mean(losses)))
        f= open("results_relu1.txt","a+")
        f.write(name1)
        f.close()
        torch.save(my_net.state_dict(), "check1.py")
        if((epoch+1) % 10 == 0):
            optim = torch.optim.SGD(my_net.parameters(), lr=0.00001, momentum=0.9,weight_decay=0.000001)
    pred = []
    for (input_val, label) in val_loader:
        prediction = my_net(to_variable(input_val))
        pred.extend(prediction.data.cpu().numpy())
    predd = np.argmax(pred, axis=1)
    
    
    write_results(predd)
if __name__ == '__main__':
    main()

(50000, 3072) (10000, 3072) (50000,)
(3072, 3072)
(50000, 3, 32, 32)
(10000, 3, 32, 32)
[0 2 1 8 1 3 7 5 1]
Epoch 0 Loss: 0.6117
:  0.8178
Epoch 1 Loss: 0.6058
:  0.8134


KeyboardInterrupt: 

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# evenly sampled time at 200ms intervals
t = np.arange(0., 5., 0.2)

# red dashes, blue squares and green triangles
plt.plot(t, t, 'r', t, t**2, 'b', t, t**3, 'g')
plt.show()